# Get Weather Station Metadata (Synoptic API)

<b>Summary:</b><br>
Uses Synoptic Weather Data API to get weather station metadata.<br>
Synoptic is partner of SDG&E, helping store and serve their weather station data--accessible via [API](https://developers.synopticdata.com/mesonet/).

- Get weather station metadata via synoptic API
- Save raw weather station metadata file

<b>Output:</b><br>
.<br>
└── data<br>
&emsp;&emsp;&emsp;└── raw<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── station_metadata.csv

<b>Instructions:</b><br>
- See README.md in directory

<b>Areas for Improvement:</b><br>

In [1]:
import json

import pandas as pd
import requests

## Parameters

In [2]:
# Base url and API token
with open("config.json") as config_file:
    cfg = json.load(config_file)

API_BASE_URL = cfg["synoptic_base_url"]
API_TOKEN = cfg["synoptic_api_token"]

# should be SHORTNAME values from mesonet
WEATHER_NETWORKS = cfg["weather_networks"]
WEATHER_NETWORKS = {int(k): v for k, v in WEATHER_NETWORKS.items()}  # convert id to int

## Get all weather station metadata for the given networks

### Helper function to pull weather station metadata using synoptic API

In [3]:
def get_station_metadata(network: str) -> dict:
    """
    Return json dictionary of station metadata within given network(s).
    """
    endpoint = f"{API_BASE_URL}stations/metadata"
    params = {
        "token": API_TOKEN,
        "network": network,
        "status": "ACTIVE",
        "obtimezone": "UTC",
        "sensorvars": 1,
        "complete": 1,
    }
    r = requests.get(endpoint, params=params)
    if r.status_code not in range(200, 300):
        # TODO: add error handling
        print("Error!")
        return {}
    return r.json()

### Get station metadata for each network

In [4]:
# convert keys to strings and commma separate
networks = ",".join([str(i) for i in list(WEATHER_NETWORKS.keys())])
stations_df = pd.json_normalize(get_station_metadata(network=networks)["STATION"])

In [5]:
stations_df

,MNET_SHORTNAME,NWSFIREZONE,ELEV_DEM,TIMEZONE,SGID,SHORTNAME,ELEVATION,STID,WIMS_ID,GACC,...,SENSOR_VARIABLES.fuel_moisture.fuel_moisture_1.PERIOD_OF_RECORD.end,SENSOR_VARIABLES.precip_accum_one_hour.precip_accum_one_hour_1.position,SENSOR_VARIABLES.precip_accum_one_hour.precip_accum_one_hour_1.PERIOD_OF_RECORD.start,SENSOR_VARIABLES.precip_accum_one_hour.precip_accum_one_hour_1.PERIOD_OF_RECORD.end,SENSOR_VARIABLES.altimeter.altimeter_1.position,SENSOR_VARIABLES.altimeter.altimeter_1.PERIOD_OF_RECORD.start,SENSOR_VARIABLES.altimeter.altimeter_1.PERIOD_OF_RECORD.end,SENSOR_VARIABLES.precip_accum_ten_minute.precip_accum_ten_minute_1.position,SENSOR_VARIABLES.precip_accum_ten_minute.precip_accum_ten_minute_1.PERIOD_OF_RECORD.start,SENSOR_VARIABLES.precip_accum_ten_minute.precip_accum_ten_minute_1.PERIOD_OF_RECORD.end
0,HPWREN,SGX258,6223.8,America/Los_Angeles,SC11,HPWREN,6181,HP001,None,SOCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HPWREN,SGX250,3700.8,America/Los_Angeles,SC08,HPWREN,3645,HP002,None,SOCC,...,2012-04-14T04:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HPWREN,SGX258,3070.9,America/Los_Angeles,SC08,HPWREN,3032,HP006,None,SOCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HPWREN,SGX258,4878.6,America/Los_Angeles,SC11,HPWREN,3580,HP009,None,SOCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HPWREN,SGX258,4235.6,America/Los_Angeles,SC11,HPWREN,4280,HP004,None,SOCC,...,NaN,NaN,2009-07-22T00:00:00Z,2013-08-14T16:52:00Z,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1772,SC-EDISON,LOX358,246.1,America/Los_Angeles,SC08,SC-EDISON,235,571SE,None,SOCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1773,SC-EDISON,HNX596,None,America/Los_Angeles,SC03,SC-EDISON,3047,572SE,None,SOCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1774,SC-EDISON,HNX596,None,America/Los_Angeles,SC03,SC-EDISON,3503,573SE,None,SOCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1775,SC-EDISON,HNX595,None,America/Los_Angeles,SC03,SC-EDISON,4126,574SE,None,SOCC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Write weather station metadata to raw folder

In [6]:
stations_df.to_csv("../../data/raw/station_metadata.csv", index=False)